## Shape Matching

**cv2.matchShapes(contour template, contour, method, method parameter)**

**Output** – match value (lower values means a closer match)

- Contour Template – This is our reference contour that we’re trying to find in the new image
- Contour – The individual contour we are checking against
- Method – Type of contour matching (1, 2, 3)
- Method Parameter – leave alone as 0.0 (not fully utilized in python OpenCV)


In [13]:
import cv2
import numpy as np


def get_center(c):
    #find the center coordinate for this circle
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    
    return cx,cy


# Load the shape template or reference image
template = cv2.imread('images/4star.jpg',0)
cv2.imshow('Template', template)
cv2.waitKey()

# Load the target image with the shapes we're trying to match
target = cv2.imread('images/shapestomatch.jpg')
target_gray = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)

# Threshold both images first before using cv2.findContours
ret, thresh1 = cv2.threshold(template, 127, 255, 0)
ret, thresh2 = cv2.threshold(target_gray, 127, 255, 0)

# Find contours in template
_, contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

# We need to sort the contours by area so that we can remove the largest
# contour which is the image outline
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)

# We extract the second largest contour which will be our template contour
template_contour = contours[1]

# Extract contours from second target image
_, contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

blank_image = np.zeros((target.shape[0], target.shape[1], 3))
#Draw all contours
cv2.drawContours(blank_image, contours, -1, (0,0,255), 3)  
cv2.imshow('Contours', blank_image)
cv2.waitKey(0)

best_match = float("inf")

for c in contours:
    # Iterate through each contour in the target image and 
    # use cv2.matchShapes to compare contour shapes
    match = cv2.matchShapes(template_contour, c, 3, 0.0)
    cx,cy = get_center(c)
    text = '{:.2%}'.format(match)
    cv2.putText(target, text, (cx - 20, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    print (match)
    # If the match value is less than 0.15 we
    if match < best_match:
        closest_contour = c
        best_match = match
                
cv2.drawContours(target, [closest_contour], -1, (0,255,0), 3)
cv2.imshow('Output', target)
cv2.waitKey()
cv2.destroyAllWindows()

0.13081816783853514
0.1590200533978871
0.1498791568252558
0.07094034474475601


http://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html